# Analyze Kaggle ELO Data

In [58]:
import pandas as panda

import numpy as np

import datetime, time

from matplotlib.pyplot import plot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns


from sklearn.linear_model import LinearRegression, RANSACRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [2]:
train_data_path = 'all/all_merged.csv'
mean_installment_path = 'all/mean_installments.csv'
max_category_1_path = 'all/max_category_1.csv'
max_category_2_path = 'all/max_category_2.csv'
max_category_3_path = 'all/max_category_3.csv'

train_data = panda.read_csv(train_data_path)

train_data.head()

,Unnamed: 0,card_id,feature_1,feature_2,feature_3,target,months_passed,card_id.1,total_no_of_transaction,total,total_accepted,total_rejects
0,0,C_ID_92A2005557,5.0,2.0,1.0,-0.820283,18.0,C_ID_415BB3A509,9,-4.135011,8,1
1,1,C_ID_3D0044924F,4.0,1.0,0.0,0.392913,23.0,C_ID_415BB3A509,9,-4.135011,8,1
2,2,C_ID_D639EDF6CD,2.0,2.0,0.0,0.688056,28.0,C_ID_415BB3A509,9,-4.135011,8,1
3,3,C_ID_186D6A6901,4.0,3.0,0.0,0.142495,15.0,C_ID_415BB3A509,9,-4.135011,8,1
4,4,C_ID_CDBD2C0DB2,1.0,3.0,0.0,-0.159749,13.0,C_ID_415BB3A509,9,-4.135011,8,1


In [3]:
mean_installment = panda.read_csv(mean_installment_path)
max_category_1 = panda.read_csv(max_category_1_path)# = 'all/max_category_1.csv'
max_category_2 = panda.read_csv(max_category_2_path)# = 'all/max_category_2.csv'
max_category_3 = panda.read_csv(max_category_3_path)# = 'all/max_category_3.csv'




print(len(train_data), len(mean_installment), len(max_category_1), len(max_category_2), len(max_category_3))

201917 325540 325540 325540 325540


In [4]:
installment_merged = train_data.merge(mean_installment, how='inner', on ='card_id')

len(installment_merged)

201917

In [5]:
max_category_merged = installment_merged.merge(max_category_1, on='card_id', how ='inner')
len(max_category_merged)

201917

In [6]:
max_category_2_merged = max_category_merged.merge(max_category_2, on='card_id', how ='inner')
len(max_category_2_merged)

201917

In [7]:
max_category_3_merged = max_category_2_merged.merge(max_category_3, on='card_id', how ='inner')
len(max_category_3_merged)

201917

In [8]:
columns_reqd = [
 'card_id',
 'feature_1',
 'feature_2',
 'feature_3',
 'target',
 'months_passed',
 'total_no_of_transaction',
 'total',
 'total_accepted',
 'total_rejects',
 'mean_installments',
 'max_cat_1',
 'max_cat_2',
 'cat_3',
]

In [9]:
# sns.pairplot(train_data[[i for i in train_data.columns.tolist() if i not in['card_id','first_active_month', 'card_id.1','Unnamed:0'] ]])

Lets concentrate on the graphs at 4th node. it is very clear that for certain values of months passed, feature 1,2,3 there are outliers affecting the target. else target is pretty much spread pretty evenly compared to the attributes


from the graph itself we can see that, target values under the range of -30 are the outliers. rest of the target values are prettu evenly placed around a particular range of -20 to 20

In [4]:
train_data.columns.tolist()

['Unnamed: 0',
 'card_id',
 'feature_1',
 'feature_2',
 'feature_3',
 'target',
 'months_passed',
 'card_id.1',
 'total_no_of_transaction',
 'total',
 'total_accepted',
 'total_rejects']

### we will proceed to run a regression model on the training data itself. using only the training data and the columns feature_1,feature_2. feature_3, and months passed as independent variables and target as a dependent variable

In [10]:
round1_training_data_ = max_category_3_merged[columns_reqd]

round1_x = round1_training_data_[[i for i in round1_training_data_.columns.tolist() if i!='target']]
round1_y = round1_training_data_.target

# round1_x_train, round1_x_test, round1_y_train, round1_y_test = train_test_split(round1_x, round1_y, test_size = 0.3, random_state = 12)

print(round1_x.shape, round1_y.shape)

(201917, 13) (201917,)


In [11]:

from math import sqrt

def root_mean_square_error(y, y_predicted):
    
    return mean_squared_error(y,y_predicted)
    
scorer = make_scorer(root_mean_square_error, greater_is_better=False)

In [59]:

classifiers = [
    LinearRegression(),
    RANSACRegressor(),
    DecisionTreeRegressor(random_state = 1, criterion = 'mse'),
    RandomForestRegressor(random_state = 1, criterion = 'mse'),
    SGDRegressor(),
#     SVR( kernel = 'rbf'),    
]


classifier_names = [
            'linear_regression',
            'ransac_regression',
            'decisiontree_regression',
            'randomforest_regression',
            'gradient_descent_regression',
#             'svr',               
    
]

classifier_param_grid = [
            
            {},
            {'ransac_regression__min_samples':[50, 75, 125, 200], 'ransac_regression__max_trials':[50, 125, 200], 'ransac_regression__residual_threshold':[5, 10, 14]},
            {'decisiontree_regression__max_depth':[6,7,8,9,10,11]},
            {'randomforest_regression__n_estimators':[1,2,3,5,6]} ,
            {'gradient_descent_regression__max_iter' : [100, 200, 300]},
#             {'svr__C':[1, 10, 20]},
    
]


    

class CodeTimer:
    
    """
        Utility custom contextual class for calculating the time 
        taken for a certain code block to execute
    
    """
    def __init__(self, name=None):
        self.name = " '"  + name + "'" if name else ''

    def __enter__(self):
        self.start = time.clock()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (time.clock() - self.start) * 1000.0
        time_taken = datetime.timedelta(milliseconds = self.took)
        print('Code block' + self.name + ' took(HH:MM:SS): ' + str(time_taken))



def runGridSearchAndPredict(pipeline, x_train, y_train, x_test, y_test, param_grid, n_jobs = 1, cv = 10, score = 'neg_mean_squared_error'):
    
    response = {}
    training_timer       = CodeTimer('training')
    testing_timer        = CodeTimer('testing')

    with training_timer:

        gridsearch = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv = cv, n_jobs = n_jobs, scoring = score)

        search = gridsearch.fit(x_train,y_train)

        print("Grid Search Best parameters ", search.best_params_)
        print("Grid Search Best score ", search.best_score_)
            
    with testing_timer:
        y_prediction = gridsearch.predict(x_test)
            
    print("Mean squared error score %s" %mean_squared_error(y_test,y_prediction))
    
    response['testing_time'] = testing_timer.took
    response['_y_prediction'] = y_prediction
    response['training_time'] = training_timer.took    
    response['mean_squared_error'] = mean_squared_error(y_test,y_prediction)
    response['root_mean_squared_error'] = search.best_score_
    response['r2_score'] = r2_score(y_test,y_prediction)
    response['best_estimator'] = search.best_estimator_
    
    return response
    


def analyzeRegressionModel(X,y):

    
    _x_train, _x_test, _y_train, _y_test = train_test_split(X, y, test_size = 0.3, random_state = 2)

    model_metrics = {}

    for model, model_name, model_param_grid in zip(classifiers, classifier_names, classifier_param_grid):

            pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    (model_name, model)
            ])

            cross_validator = KFold(n_splits = 10, random_state = 12)    
            result = runGridSearchAndPredict(pipeline, _x_train, _y_train, _x_test, _y_test, model_param_grid, cv =cross_validator,score = scorer)

            _y_prediction = result['_y_prediction']

            model_metrics[model_name] = {}
            model_metrics[model_name]['training_time'] = result['training_time']
            model_metrics[model_name]['testing_time'] = result['testing_time']
            model_metrics[model_name]['r2_score'] = result['r2_score']
            model_metrics[model_name]['mean_squared_error'] = result['mean_squared_error']
            model_metrics[model_name]['root_mean_squared_error'] = result['root_mean_squared_error']
            model_metrics[model_name]['best_estimator'] = result['best_estimator']
            
    return model_metrics
    print('Model metrics are \n :', model_metrics)


In [17]:
round1_x['max_cat_1'] = round1_x.max_cat_1.apply(lambda x: 1 if x=='N' else 0)

round1_x.head()

,card_id,feature_1,feature_2,feature_3,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3,
0,C_ID_92A2005557,5.0,2.0,1.0,18.0,9,-4.135011,8,1,0.014134,1,5.0,3,1
1,C_ID_3D0044924F,4.0,1.0,0.0,23.0,9,-4.135011,8,1,1.542135,0,1.0,3,0
2,C_ID_D639EDF6CD,2.0,2.0,0.0,28.0,9,-4.135011,8,1,0.000000,1,5.0,3,1
3,C_ID_186D6A6901,4.0,3.0,0.0,15.0,9,-4.135011,8,1,1.059524,0,4.0,3,0
4,C_ID_CDBD2C0DB2,1.0,3.0,0.0,13.0,9,-4.135011,8,1,1.284024,0,5.0,3,0


In [24]:
columns = [i for i in columns_reqd if i!='card_id' and i!='target']

round1_x = round1_x[columns]

round1_x.head()

,feature_1,feature_2,feature_3,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
0,5.0,2.0,1.0,18.0,9,-4.135011,8,1,0.014134,1,5.0,3
1,4.0,1.0,0.0,23.0,9,-4.135011,8,1,1.542135,0,1.0,3
2,2.0,2.0,0.0,28.0,9,-4.135011,8,1,0.000000,1,5.0,3
3,4.0,3.0,0.0,15.0,9,-4.135011,8,1,1.059524,0,4.0,3
4,1.0,3.0,0.0,13.0,9,-4.135011,8,1,1.284024,0,5.0,3


In [25]:
model_metrics = analyzeRegressionModel(round1_x, round1_y)

Grid Search Best parameters  {}
Grid Search Best score  -14.695391244138122
Code block 'training' took(HH:MM:SS): 0:00:01.881956
Code block 'testing' took(HH:MM:SS): 0:00:00.012446
Mean squared error score 14.897911103411596
Grid Search Best parameters  {'ransac_regression__max_trials': 50, 'ransac_regression__min_samples': 50, 'ransac_regression__residual_threshold': 5}
Grid Search Best score  -14.894202561147784
Code block 'training' took(HH:MM:SS): 0:02:15.275676
Code block 'testing' took(HH:MM:SS): 0:00:00.012421
Mean squared error score 15.129175948887196
Grid Search Best parameters  {'decisiontree_regression__max_depth': 6}
Grid Search Best score  -14.760760542081176
Code block 'training' took(HH:MM:SS): 0:00:31.148675
Code block 'testing' took(HH:MM:SS): 0:00:00.032497
Mean squared error score 14.898050658977798
Grid Search Best parameters  {'randomforest_regression__n_estimators': 6}
Grid Search Best score  -18.89215174114171
Code block 'training' took(HH:MM:SS): 0:02:27.972114

In [26]:
best_estimator = model_metrics['linear_regression']['best_estimator']

In [27]:
best_estimator


## our round_1 estimator is ready.we are simply considering the training data verbatim.

## before we predict on our test data, we would have to read it and then convert the active_first value as well.
## independent variables will be feature_1,_2,_3, months_passed

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [57]:
from dateutil import relativedelta


def get_months_passed( given):
    
    current = datetime.datetime.now()
    given = datetime.datetime.strptime(given + '-01', '%Y-%m-%d')
    
    r = relativedelta.relativedelta(current, given)
    return r.months + r.years*12

test_data = panda.read_csv('all/test_all_merged.csv')

# test_data['first_active_month'].fillna(test_data['first_active_month'].value_counts().idxmax(), inplace= True)

# test_data['months_passed'] = test_data['first_active_month'].apply(lambda x: get_months_passed(x))
test_data.head()

# test_x = test_data[[i for i in test_data.columns.tolist() if i not in  ['card_id','first_active_month', 'card_id.1','Unnamed: 0'] ]]
# test_x.head()



,Unnamed: 0,card_id,feature_1,feature_2,feature_3,months_passed,card_id.1,total_no_of_transaction,total,total_accepted,total_rejects
0,0,C_ID_0AB67A22AB,3.0,3.0,1.0,20.0,C_ID_415BB3A509,9,-4.135011,8,1
1,1,C_ID_130FD0CBDD,2.0,3.0,0.0,23.0,C_ID_415BB3A509,9,-4.135011,8,1
2,2,C_ID_B709037BC5,5.0,1.0,1.0,16.0,C_ID_415BB3A509,9,-4.135011,8,1
3,3,C_ID_D27D835A9F,2.0,1.0,0.0,12.0,C_ID_415BB3A509,9,-4.135011,8,1
4,4,C_ID_2B5E3DF5C2,5.0,1.0,1.0,36.0,C_ID_415BB3A509,9,-4.135011,8,1


In [39]:
test_installment_merged = test_data.merge(mean_installment, how='inner', on ='card_id')


test_max_category_merged = test_installment_merged.merge(max_category_1, on='card_id', how ='inner')


test_max_category_2_merged = test_max_category_merged.merge(max_category_2, on='card_id', how ='inner')


test_max_category_3_merged = test_max_category_2_merged.merge(max_category_3, on='card_id', how ='inner')



print(len(test_data),len(test_installment_merged),len(test_max_category_merged), len(test_max_category_2_merged),len(test_max_category_3_merged))


123623 123623 123623 123623 123623


In [40]:
# test_x = test_max_category_3_merged[columns]

test_max_category_3_merged.head()

,Unnamed: 0_x,card_id,feature_1,feature_2,feature_3,months_passed,card_id.1,total_no_of_transaction,total,total_accepted,...,installments_y,category_1_x,category_3_y,cat_3_y,Unnamed: 0,cat_3,installments,category_1_y,category_2_y,category_3
0,0,C_ID_0AB67A22AB,3.0,3.0,1.0,20.0,C_ID_415BB3A509,9,-4.135011,8,...,71,71,71,71,313527,2,48,48,48,48
1,1,C_ID_130FD0CBDD,2.0,3.0,0.0,23.0,C_ID_415BB3A509,9,-4.135011,8,...,82,82,82,82,345979,2,84,84,84,84
2,2,C_ID_B709037BC5,5.0,1.0,1.0,16.0,C_ID_415BB3A509,9,-4.135011,8,...,12,12,12,12,596925,3,3,3,3,3
3,3,C_ID_D27D835A9F,2.0,1.0,0.0,12.0,C_ID_415BB3A509,9,-4.135011,8,...,1,1,1,1,469022,3,2,2,2,2
4,4,C_ID_2B5E3DF5C2,5.0,1.0,1.0,36.0,C_ID_415BB3A509,9,-4.135011,8,...,106,106,106,106,555587,3,1,1,1,1


In [47]:
cols = [ 'feature_1',
 'feature_2',
 'feature_3',
 'months_passed',
 'total_no_of_transaction',
 'total',
 'total_accepted',
 'total_rejects',
 'mean_installments',
 'max_cat_1',
 'max_cat_2',
 'cat_3',]

    
test_x = test_max_category_3_merged[cols]


test_x['max_cat_1'] = test_x.max_cat_1.apply(lambda x: 1 if x=='N' else 0)
test_x.head()

C:\Users\somak\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,feature_1,feature_2,feature_3,months_passed,total_no_of_transaction,total,total_accepted,total_rejects,mean_installments,max_cat_1,max_cat_2,cat_3
0,3.0,3.0,1.0,20.0,9,-4.135011,8,1,2.056338,0,1.0,2
1,2.0,3.0,0.0,23.0,9,-4.135011,8,1,1.080460,0,4.0,2
2,5.0,1.0,1.0,16.0,9,-4.135011,8,1,3.666667,0,5.0,3
3,2.0,1.0,0.0,12.0,9,-4.135011,8,1,1.861111,0,5.0,3
4,5.0,1.0,1.0,36.0,9,-4.135011,8,1,1.077586,1,4.0,3


In [48]:

test_target = best_estimator.predict(test_x)

test_target

array([-0.69919571, -0.74089561, -0.32047768, ..., -0.60463083,
       -0.30425776, -0.5765708 ])

In [52]:
## write it to a panda table

temp = panda.read_csv('all/test.csv')
sample_submission_round_1 = temp[['card_id']]

# sample_submission_round_1 = test_data[['card_id']]
sample_submission_round_1['target'] = test_target


print(len(sample_submission_round_1), sample_submission_round_1.head())

sample_submission_round_1.to_csv('round7_submission.csv')

C:\Users\somak\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


123623            card_id    target
0  C_ID_0ab67a22ab -0.699196
1  C_ID_130fd0cbdd -0.740896
2  C_ID_b709037bc5 -0.320478
3  C_ID_d27d835a9f -0.187437
4  C_ID_2b5e3df5c2 -0.493469


In [56]:
test_x.to_csv('all/my_test_data_categorized_can_be_used_as_is.csv')
round1_training_data_.to_csv('all/my_training_data_can_be_used_as_is.csv')

##### lets try and see if we add a cross validator and re run

In [60]:
model_metrics = analyzeRegressionModel(round1_x, round1_y)

Grid Search Best parameters  {}
Grid Search Best score  -14.695391244138122
Code block 'training' took(HH:MM:SS): 0:00:01.649316
Code block 'testing' took(HH:MM:SS): 0:00:00.013773
Mean squared error score 14.897911103411596
Grid Search Best parameters  {'ransac_regression__max_trials': 50, 'ransac_regression__min_samples': 50, 'ransac_regression__residual_threshold': 14}
Grid Search Best score  -14.897859094650636
Code block 'training' took(HH:MM:SS): 0:02:29.322484
Code block 'testing' took(HH:MM:SS): 0:00:00.014839
Mean squared error score 15.111070587140123
Grid Search Best parameters  {'decisiontree_regression__max_depth': 6}
Grid Search Best score  -14.760760542081176
Code block 'training' took(HH:MM:SS): 0:00:32.760737
Code block 'testing' took(HH:MM:SS): 0:00:00.016704
Mean squared error score 14.898050658977798
Grid Search Best parameters  {'randomforest_regression__n_estimators': 6}
Grid Search Best score  -18.89215174114171
Code block 'training' took(HH:MM:SS): 0:02:31.86170

In [61]:

best_estimator = model_metrics['linear_regression']['best_estimator']
test_target = best_estimator.predict(test_x)

test_target
temp = panda.read_csv('all/test.csv')
sample_submission_round_1 = temp[['card_id']]

# sample_submission_round_1 = test_data[['card_id']]
sample_submission_round_1['target'] = test_target


print(len(sample_submission_round_1), sample_submission_round_1.head())

sample_submission_round_1.to_csv('round_linear_Regression_with_cv_submission.csv')

C:\Users\somak\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


123623            card_id    target
0  C_ID_0ab67a22ab -0.699196
1  C_ID_130fd0cbdd -0.740896
2  C_ID_b709037bc5 -0.320478
3  C_ID_d27d835a9f -0.187437
4  C_ID_2b5e3df5c2 -0.493469
